<a href="https://colab.research.google.com/github/TamandaCodes/BrodgarFNO/blob/main/Brodgar_Transient_FNO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# MOUNT THE DRIVE!!!
import os, sys
from google.colab import drive

drive_name = '/content/gdrive'
if os.path.ismount(drive_name):
    print(f'Drive ({drive_name}) already mounted')
else:
    drive.mount(drive_name)

# Project dir (where artifacts & outputs will be written)
dir_path = '/content/gdrive/MyDrive/Colab Notebooks/FourierNeuralOperator/Transient Folder'
os.makedirs(dir_path, exist_ok=True)
sys.path.insert(1, dir_path)

Drive (/content/gdrive) already mounted


In [ ]:
# fno_full_block_train_infer.py
import os
import math
import torch
import torch.nn as nn
import torch.fft as fft
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, random_split
from typing import Tuple, Dict, Any
import numpy as np
from data_store_load import load_full_dataset
from fno_implementation_modified import (
    compute_norm_stats, FNO2DSpacetime, OneStepDataset,
    collate_batch, train_one_epoch, eval_one_epoch, autoregressive_rollout
)

In [ ]:
# --- imports you rely on (make sure these exist up top) ---
import os, math, torch, torch.nn as nn
from torch.utils.data import DataLoader

# =========================
# Root dir & path helpers
# =========================
try:
    DIR = dir_path
except Exception:
    DIR = ""

def P(fn: str) -> str:
    "Join to project directory for outputs/artifacts."
    return os.path.join(DIR, fn)

def find_input(filename: str) -> str:
    cands = [
        os.path.join(DIR, filename),
        os.path.join('/mnt/data', filename),
        filename,  # absolute / custom
    ]
    for p in cands:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Could not find input file '{filename}' in {cands}")

# Checkpoint path
ckpt_path = P("fno_autoregressive.pt")

In [ ]:
import torch
import torch.nn as nn

class H1Loss(nn.Module):
    """
    Sobolev Loss (H1): Penalizes errors in value AND errors in spatial slope (derivative).
    Use this to force the model to capture high-frequency 'wiggles'.
    """
    def __init__(self, alpha=0.1):
        super().__init__()
        self.alpha = alpha  # Weight for the derivative term
        self.l1 = nn.L1Loss() # Base loss (L1 is sharper than MSE)

    def forward(self, pred, target):
        # 1. Standard L1 Loss on values (Magnitude accuracy)
        loss_val = self.l1(pred, target)

        # 2. L1 Loss on Spatial Gradients (Shape/Wiggle accuracy)
        # Calculate gradients along the spatial dimension (last dim, X)
        # Input shape is usually [B, C, T, X] or [B, C, 1, X]
        pred_dx = pred[..., 1:] - pred[..., :-1]
        target_dx = target[..., 1:] - target[..., :-1]

        loss_grad = self.l1(pred_dx, target_dx)

        # Total Loss
        return loss_val + (self.alpha * loss_grad)

## MAIN

In [ ]:
# =========================
#           MAIN
# =========================
def main():
    print("="*80)
    print("FNO 1D SPACE PIPELINE TRANSIENT MODEL")
    print("="*80)

   # ---- Configuration ----
    IN_CHANNELS = 10   # All 10 variables as input
    OUT_CHANNELS = 7   # First 7 variables as output

    # ---- paths ----
    dataset_file = find_input("cleaned_tensor.pt")
    print(f"\n1. Configuration")
    print(f"   Dataset file: {dataset_file}")
    print(f"   Input channels: {IN_CHANNELS}")
    print(f"   Output channels: {OUT_CHANNELS}")

    # ---- check if dataset exists ----
    if not os.path.exists(dataset_file):
        print(f"\n❌ Error: Dataset file not found: {dataset_file}")
        print("   Please run data_store_load.py first to create the dataset.")
        return # Return None if dataset not found

     # ---- load dataset ----
    print(f"\n2. Loading dataset...")
    try:
        # Load your tensor format
        data = torch.load(dataset_file)

        # Handle different possible formats
        if isinstance(data, dict):
            full_tensor = data['full_tensor']  # [50, 1279, 10, 288]
            variable_names = data.get('variable_names', [f'Var{i}' for i in range(IN_CHANNELS)])
        else:
            full_tensor = data
            variable_names = [f'Var{i}' for i in range(IN_CHANNELS)]

        N, X, C, T = full_tensor.shape

        print(f"   ✓ Dataset shape: {full_tensor.shape}")
        print(f"   ✓ Cases: {N}")
        print(f"   ✓ Sections: {X}")
        print(f"   ✓ Variables: {C}")
        print(f"   ✓ Timesteps: {T}")
        print(f"   ✓ Variable names: {variable_names}")

        # Validate dimensions
        assert C >= IN_CHANNELS, f"Expected at least {IN_CHANNELS} channels, got {C}"
        assert C >= OUT_CHANNELS, f"Expected at least {OUT_CHANNELS} channels, got {C}"

    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        return # Return None if error loading dataset

    # Create section lengths (uniform or load actual if available)
    section_lengths = np.arange(X, dtype=np.float32)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"   ✓ Using device: {device}")

    # ---- splits (by case) ----
    print(f"\n3. Creating train/val/test splits...")
    N = full_tensor.shape[0]
    n_train = int(0.8 * N)
    n_val   = int(0.1 * N)
    n_test  = N - n_train - n_val

    print(f"   Total cases: {N}")
    print(f"   Train: {n_train} cases ({100*n_train/N:.0f}%)")
    print(f"   Val: {n_val} cases ({100*n_val/N:.0f}%)")
    print(f"   Test: {n_test} cases ({100*n_test/N:.0f}%)")

    # simple contiguous split; switch to random if you prefer
    train_tensor = full_tensor[:n_train]
    val_tensor   = full_tensor[n_train:n_train+n_val]
    test_tensor  = full_tensor[n_train+n_val:]

     # ---- normalization on train only ----
    print(f"\n4. Computing normalization statistics...")
    mean_input, std_input, mean_output, std_output = compute_norm_stats(
        train_tensor, IN_CHANNELS, OUT_CHANNELS
    )

    print("   Input variables (10):")
    for i in range(IN_CHANNELS):
        var_name = variable_names[i] if i < len(variable_names) else f'Var{i}'
        print(f"     {var_name}: mean={mean_input[i]:.3e}, std={std_input[i]:.3e}")

    print("   Output variables (7):")
    for i in range(OUT_CHANNELS):
        var_name = variable_names[i] if i < len(variable_names) else f'Var{i}'
        print(f"     {var_name}: mean={mean_output[i]:.3e}, std={std_output[i]:.3e}")


    # ---- datasets & loaders ----
    print(f"\n5. Creating datasets and dataloaders...")

    # IMPORTANT: your dataset already returns CUDA tensors when device="cuda".
    # To avoid the pin_memory crash, either:
    #   A) keep device=cuda and DISABLE pin_memory, or
    #   B) force dataset to return CPU tensors and move to device in the loop.
    # We'll do (A) for minimal change.
    train_ds = OneStepDataset(
        train_tensor, section_lengths,
        mean_input, std_input, mean_output, std_output,
        IN_CHANNELS, OUT_CHANNELS, device=device
    )
    val_ds = OneStepDataset(
        val_tensor, section_lengths,
        mean_input, std_input, mean_output, std_output,
        IN_CHANNELS, OUT_CHANNELS, device=device
    )
    test_ds = OneStepDataset(
        test_tensor, section_lengths,
        mean_input, std_input, mean_output, std_output,
        IN_CHANNELS, OUT_CHANNELS, device=device
    )

    print(f"   Train samples: {len(train_ds)} ({n_train} cases × {T-1} timesteps)")
    print(f"   Val samples: {len(val_ds)}")
    print(f"   Test samples: {len(test_ds)}")

    batch_size = 32 if device == "cuda" else 4
    print(f"   Batch size: {batch_size}")

    # If tensors are already on CUDA, pin_memory must be False (pinning works only for CPU tensors).
    use_pin_memory = False

    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True, num_workers=0,
        pin_memory=use_pin_memory, collate_fn=collate_batch
    )
    val_loader   = DataLoader(
        val_ds,   batch_size=batch_size, shuffle=False, num_workers=0,
        pin_memory=use_pin_memory, collate_fn=collate_batch
    )

    # ---- model ----
   # in_ch = IN_CHANNELS + 3 (t_coord, x_coord, t0_mask)
    model_in_ch = IN_CHANNELS + 3

    model = FNO2DSpacetime(
        in_ch=model_in_ch,
        out_ch=OUT_CHANNELS,
        width=256, #Changed from 128 -> 256
        depth=5,
        k_t=1,   # KEY: Single mode in time (essentially spatial-only)
        k_x=128,  # Spatial modes; Changed from 64 -> 128
        predict_delta_from_u0=False
    ).to(device)

    n_params = sum(p.numel() for p in model.parameters())
    print(f"   ✓ Model created")
    print(f"   ✓ Parameters: {n_params:,}")
    print(f"   ✓ Input channels: {model_in_ch} ({IN_CHANNELS} vars + 3 coords)")
    print(f"   ✓ Output channels: {OUT_CHANNELS}")
    print(f"   ✓ k_t: 1 (spatial-only)")
    print(f"   ✓ k_x: 96")

    opt = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
    loss_fn = H1Loss(alpha=0.5)
    sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.5, patience=5)

    # Fix deprecated GradScaler usage
    scaler = torch.amp.GradScaler('cuda') if device == "cuda" else None

    # ---- training loop with early stopping ----
    max_epochs = 100
    early_stopping_patience = 10

    print(f"\n7. Starting training...")
    print(f"   Max epochs: {max_epochs}")
    print(f"   Early stopping patience: {early_stopping_patience}")
    print(f"   Learning rate: 1e-3")
    print("-" * 80)

    best_val = math.inf
    bad = 0

    import time
    start_time = time.time()

    train_losses = []
    val_losses = []

    for epoch in range(1, max_epochs + 1):
        epoch_start = time.time()
        print(f"\n[Epoch {epoch:03d}/{max_epochs}]")

        tr = train_one_epoch(model, train_loader, opt, loss_fn, scaler=scaler)
        va = eval_one_epoch(model, val_loader, loss_fn)

        train_losses.append(tr)
        val_losses.append(va)

        epoch_time = time.time() - epoch_start
        current_lr = opt.param_groups[0]['lr']

        sched.step(va)

        improvement = ""
        if va + 1e-8 < best_val:
            improvement = f" ⭐ NEW BEST! (improved by {(best_val - va):.6f})"

        print(f"  Results: Train Loss = {tr:.6f} | Val Loss = {va:.6f}")
        print(f"  Time: {epoch_time:.1f}s | LR: {current_lr:.2e}{improvement}")

        if va + 1e-8 < best_val:
            best_val = va
            bad = 0
            # Save checkpoint
            os.makedirs(os.path.dirname(ckpt_path) or ".", exist_ok=True)
            torch.save({
                "model": model.state_dict(),
                "mean_input": mean_input,
                "std_input": std_input,
                "mean_output": mean_output,
                "std_output": std_output,
                "variable_names": variable_names,
                "section_lengths": torch.tensor(section_lengths, dtype=torch.float32),
                "in_channels": IN_CHANNELS,
                "out_channels": OUT_CHANNELS,
                "config": {
                    "width": 256,
                    "depth": 5,
                    "k_t": 1,
                    "k_x": 128,
                }
            }, ckpt_path)
            print(f"  ✓ Checkpoint saved to: {ckpt_path}")
        else:
            bad += 1
            print(f"  No improvement ({bad}/{early_stopping_patience})")
            if bad >= early_stopping_patience:
                total_time = time.time() - start_time
                print(f"\n  ⚠ Early stopping triggered after {epoch} epochs ({total_time/60:.1f} minutes)")
                break

    total_time = time.time() - start_time
    print(f"\n" + "=" * 80)
    print(f"TRAINING COMPLETE!")
    print(f"  Total time: {total_time/60:.1f} minutes")
    print(f"  Best validation loss: {best_val:.6f}")
    print(f"  Final epoch: {epoch}")
    print("=" * 80)

    # ---- inference demo on one test case ----
    """if len(test_tensor) > 0:
        case0 = test_tensor[0]  # [X,5,121]
        ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
        model.load_state_dict(ckpt["model"])
        model.eval()
        preds = predict_full_block(model, case0, section_lengths, mean, std, device=device)  # [5,120,X]
        var_idx = variable_names.index("PT") if "PT" in variable_names else 0
        pt_pred = preds[var_idx]  # [120,X]
        print(f"Inference OK. PT prediction block: mean={pt_pred.mean().item():.4f}, std={pt_pred.std().item():.4f}")"""


    import matplotlib.pyplot as plt
    # ---- Plot training curves ----
    print(f"\n8. Plotting training curves...")
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Val Loss')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.grid(True)
    plt.yscale('log')
    curves_path = P('training_curves.png')
    plt.savefig(curves_path, dpi=150)
    print(f"  ✓ Saved: {curves_path}")
    plt.close()





if __name__ == "__main__":
    # Call main
    main()

FNO 1D SPACE PIPELINE TRANSIENT MODEL

1. Configuration
   Dataset file: /content/gdrive/MyDrive/Colab Notebooks/FourierNeuralOperator/Transient Folder/cleaned_tensor.pt
   Input channels: 10
   Output channels: 7

2. Loading dataset...
   ✓ Dataset shape: torch.Size([50, 1279, 10, 288])
   ✓ Cases: 50
   ✓ Sections: 1279
   ✓ Variables: 10
   ✓ Timesteps: 288
   ✓ Variable names: ['PT', 'TM', 'HOL', 'HOLWT', 'UG', 'UL', 'ULWT', 'TT', 'PI', 'FI']
   ✓ Using device: cuda

3. Creating train/val/test splits...
   Total cases: 50
   Train: 40 cases (80%)
   Val: 5 cases (10%)
   Test: 5 cases (10%)

4. Computing normalization statistics...
   Input variables (10):
     PT: mean=5.287e+01, std=1.649e+01
     TM: mean=3.784e+01, std=1.281e+01
     HOL: mean=1.255e-01, std=3.184e-02
     HOLWT: mean=7.121e-02, std=3.003e-02
     UG: mean=7.286e+00, std=2.392e+00
     UL: mean=2.443e+00, std=9.425e-01
     ULWT: mean=2.046e+00, std=9.085e-01
     TT: mean=6.081e+01, std=5.457e+00
     PI: mean

# Inference and Comparison using Trained Model

In [ ]:
import os
import math
import torch
import torch.nn as nn
import pandas as pd
import torch.fft as fft
from torch.utils.data import Dataset, DataLoader, random_split
import matplotlib.pyplot as plt
from typing import Tuple, Dict, Any
import numpy as np
from data_store_load import load_full_dataset
from fno_implementation_modified import (
    compute_norm_stats, FNO2DSpacetime, OneStepDataset,
    collate_batch, train_one_epoch, eval_one_epoch, autoregressive_rollout
)

In [ ]:
# --- imports you rely on (make sure these exist up top) ---
import os, math, torch, torch.nn as nn
from torch.utils.data import DataLoader

# =========================
# Root dir & path helpers
# =========================
try:
    DIR = dir_path
except Exception:
    DIR = ""

def P(fn: str) -> str:
    "Join to project directory for outputs/artifacts."
    return os.path.join(DIR, fn)

def find_input(filename: str) -> str:
    cands = [
        os.path.join(DIR, filename),
        os.path.join('/mnt/data', filename),
        filename,  # absolute / custom
    ]
    for p in cands:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Could not find input file '{filename}' in {cands}")

# Checkpoint path
ckpt_path = P("fno_autoregressive.pt")

In [ ]:
def main():
    print("="*80)
    print("FNO 1D SPACE PIPELINE TRANSIENT MODEL INFERENCE")
    print("="*80)

   # ---- Configuration ----
    IN_CHANNELS = 10   # All 10 variables as input
    OUT_CHANNELS = 7   # First 7 variables as output

    # ---- paths ----
    dataset_file = find_input("cleaned_tensor.pt")
    print(f"\n1. Configuration")
    print(f"   Dataset file: {dataset_file}")
    print(f"   Input channels: {IN_CHANNELS}")
    print(f"   Output channels: {OUT_CHANNELS}")

    # ---- check if dataset exists ----
    if not os.path.exists(dataset_file):
        print(f"\n❌ Error: Dataset file not found: {dataset_file}")
        print("   Please run data_store_load.py first to create the dataset.")
        return # Return None if dataset not found

     # ---- load dataset ----
    print(f"\n2. Loading dataset...")
    try:
        # Load your tensor format
        data = torch.load(dataset_file)

        # Handle different possible formats
        if isinstance(data, dict):
            full_tensor = data['full_tensor']  # [50, 1279, 10, 288]
            variable_names = data.get('variable_names', [f'Var{i}' for i in range(IN_CHANNELS)])
        else:
            full_tensor = data
            variable_names = [f'Var{i}' for i in range(IN_CHANNELS)]

        N, X, C, T = full_tensor.shape

        print(f"   ✓ Dataset shape: {full_tensor.shape}")
        print(f"   ✓ Cases: {N}")
        print(f"   ✓ Sections: {X}")
        print(f"   ✓ Variables: {C}")
        print(f"   ✓ Timesteps: {T}")
        print(f"   ✓ Variable names: {variable_names}")

        # Validate dimensions
        assert C >= IN_CHANNELS, f"Expected at least {IN_CHANNELS} channels, got {C}"
        assert C >= OUT_CHANNELS, f"Expected at least {OUT_CHANNELS} channels, got {C}"

    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        return # Return None if error loading dataset

    # Convert section_lengths to numpy if it's a tensor
    #if isinstance(section_lengths, torch.Tensor):
    #    section_lengths = section_lengths.numpy()

    # Create section lengths (uniform or load actual if available)
    section_lengths = np.arange(X, dtype=np.float32)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"   ✓ Using device: {device}")

    # ---- splits (by case) ----
    print(f"\n3. Creating train/val/test splits...")
    N = full_tensor.shape[0]
    n_train = int(0.8 * N)
    n_val   = int(0.1 * N)
    n_test  = N - n_train - n_val

    print(f"   Total cases: {N}")
    print(f"   Train: {n_train} cases ({100*n_train/N:.0f}%)")
    print(f"   Val: {n_val} cases ({100*n_val/N:.0f}%)")
    print(f"   Test: {n_test} cases ({100*n_test/N:.0f}%)")

    # simple contiguous split; switch to random if you prefer
    train_tensor = full_tensor[:n_train]
    val_tensor   = full_tensor[n_train:n_train+n_val]
    test_tensor  = full_tensor[n_train+n_val:]

     # in_ch = IN_CHANNELS + 3 (t_coord, x_coord, t0_mask)
    model_in_ch = IN_CHANNELS + 3

    # Normalization
    mean_input, std_input, mean_output, std_output = compute_norm_stats(
        train_tensor, IN_CHANNELS, OUT_CHANNELS
    )
    # Move normalization stats to the correct device
    mean_input = mean_input.to(device)
    std_input = std_input.to(device)
    mean_output = mean_output.to(device)
    std_output = std_output.to(device)

    model = FNO2DSpacetime(
        in_ch=model_in_ch,
        out_ch=OUT_CHANNELS,
        width=256, #Changed from 128 -> 256
        depth=5,
        k_t=1,   # KEY: Single mode in time (essentially spatial-only)
        k_x=128,  # Spatial modes : Changed from 64 -> 128
        predict_delta_from_u0=False
    ).to(device)


    # ---- Autoregressive inference on test case ----
    print(f"\n Testing autoregressive rollout...")
    if len(test_tensor) > 0:
      # Load best model
      ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
      model.load_state_dict(ckpt["model"])
      model.eval()

      # Test on first test case
      test_case = test_tensor[0]  # [X, C, T]
      initial_state = test_case[:, :, 0]  # [X, C]
      ground_truth = test_case  # [X, C, T]

      #n_rollout_steps = min(100, T-1)  # Rollout for 100 steps or less
      n_rollout_steps = T-1

      print(f"  Rolling out {n_rollout_steps} timesteps...")
      predictions = autoregressive_rollout(
          model, initial_state, ground_truth, section_lengths,
          mean_input, std_input, mean_output, std_output,
          IN_CHANNELS, OUT_CHANNELS, n_rollout_steps, device=device
      )

      # Calculate errors
      gt = ground_truth[:, :OUT_CHANNELS, 1:n_rollout_steps+1].permute(1, 2, 0)  # [C_out, n_steps, X]
      mse = ((predictions - gt) ** 2).mean()
      mae = (predictions - gt).abs().mean()

      print(f"\n11. Plotting SPATIAL profiles at specific timesteps...")

      # --- Define timesteps of interest ---
      # (t=0 is the first prediction, t=99 is the last)
      time_indices_to_plot = [0, 29, 99]
      spatial_axis = np.arange(X) # X = 1279 sections

      for t_idx in time_indices_to_plot:
          # One figure per timestep, showing all 7 variables
          fig, axes = plt.subplots(OUT_CHANNELS, 1, figsize=(14, 16), sharex=True)
          fig.suptitle(f'Spatial Profile Comparison at Timestep t={t_idx+1}', fontsize=16)

          for var_idx in range(OUT_CHANNELS):
              ax = axes[var_idx]

              # Get the spatial slice for this variable at this time
              # Shape: [X]
              pred_spatial = predictions[var_idx, t_idx, :].cpu().numpy()
              gt_spatial   = gt[var_idx, t_idx, :].cpu().numpy()

              ax.plot(spatial_axis, gt_spatial, label='Ground Truth', linewidth=1.5, alpha=0.8)
              ax.plot(spatial_axis, pred_spatial, label='Prediction', linestyle='--', linewidth=1.5, alpha=0.8)

              var_name = variable_names[var_idx] if var_idx < len(variable_names) else f'Var{var_idx}'
              ax.set_title(var_name, fontweight='bold')
              ax.set_ylabel('Value')
              ax.legend(loc='best')
              ax.grid(True, alpha=0.3)

          axes[-1].set_xlabel('Spatial Section (X)')
          plt.tight_layout(rect=[0, 0.03, 1, 0.95])
          spatial_plot_path = P(f'spatial_profile_t{t_idx+1}.png')
          plt.savefig(spatial_plot_path, dpi=150)
          print(f"  ✓ Saved spatial plot: {spatial_plot_path}")
          plt.close(fig)

      print(f"  ✓ Rollout complete")
      print(f"  Test MSE: {mse:.6f}")
      print(f"  Test MAE: {mae:.6f}")

      # Plot predictions for middle section
      print(f"\n10. Plotting predictions...")
      section_idx = X // 2

      fig, axes = plt.subplots(4, 2, figsize=(14, 14))
      fig.suptitle(f'Autoregressive Predictions vs Ground Truth\nSection {section_idx}', fontsize=14)

      for var_idx in range(min(7, OUT_CHANNELS)):
          row = var_idx // 2
          col = var_idx % 2

          if var_idx == 6:
              row, col = 3, 0

          ax = axes[row, col]

          pred = predictions[var_idx, :, section_idx].cpu().numpy()
          truth = gt[var_idx, :, section_idx].cpu().numpy()
          timesteps = np.arange(1, len(pred)+1)

          ax.plot(timesteps, truth, label='Ground Truth', linewidth=1.5, alpha=0.7)
          ax.plot(timesteps, pred, label='Prediction', linestyle='--', linewidth=1.5, alpha=0.7)

          var_name = variable_names[var_idx] if var_idx < len(variable_names) else f'Var{var_idx}'
          ax.set_title(var_name, fontweight='bold')
          ax.set_xlabel('Timestep')
          ax.set_ylabel('Value')
          ax.legend(loc='best')
          ax.grid(True, alpha=0.3)

      # Hide empty subplot
      axes[3, 1].axis('off')

      plt.tight_layout()
      pred_path = P('autoregressive_predictions.png')
      plt.savefig(pred_path, dpi=150)
      print(f"  ✓ Saved: {pred_path}")
      plt.close()

      # # ... (after your plotting code) ...

      # # ==========================================================
      # #    NEW: EXPORT COMPARISON FOR SECTIONS 1 AND 1279
      # # ==========================================================
      # print(f"\n12. Exporting comparison data for all 10 variables...")

      # # --- Parameters ---
      # sections_to_export = [0, 1278] # Section 1 (index 0) and 1279 (index 1278)
      # case_to_export = 1

      # # Get the variable names for all 10 columns
      # all_var_names = [variable_names[i] for i in range(IN_CHANNELS)]

      # # Define the output path
      # excel_path = P(f"Case{case_to_export}_edge_sections_ALL_VARS.xlsx")

      # # Use ExcelWriter to save multiple sheets in one file
      # with pd.ExcelWriter(excel_path) as writer:
      #     print(f"  Saving to: {excel_path}")

      #     for section_idx in sections_to_export:
      #         section_num = section_idx + 1
      #         print(f"    - Processing Section {section_num}...")

      #         # --- 1. Slice all Tensors ---

      #         # A. Get the 7 PREDICTED variables (shape [100, 7])
      #         pred_slice_7 = predictions[:, :, section_idx].permute(1, 0).cpu().numpy()

      #         # B. Get the 7 GROUND TRUTH variables (shape [100, 7])
      #         gt_slice_7   = gt[:, :, section_idx].permute(1, 0).cpu().numpy()

      #         # C. Get the 3 "CHEATER" variables from the *original* ground_truth tensor
      #         #    Shape is [X, C, T] -> [1279, 10, 288]
      #         #    We slice it: [section, channels 7-9, timesteps 1-100]
      #         cheater_slice = ground_truth[section_idx, OUT_CHANNELS:IN_CHANNELS, 1:n_rollout_steps+1]
      #         # -> Shape is [3, 100], permute to [100, 3]
      #         cheater_slice_np = cheater_slice.permute(1, 0).cpu().numpy()

      #         # --- 2. Combine and Create Pandas DataFrames ---

      #         # Combine 7 predicted + 3 cheater
      #         pred_combined_10 = np.concatenate([pred_slice_7, cheater_slice_np], axis=1)
      #         df_pred = pd.DataFrame(pred_combined_10, columns=all_var_names)
      #         df_pred.insert(0, 'Timestep', np.arange(1, n_rollout_steps + 1))
      #         df_pred.insert(0, 'DataType', 'Prediction')

      #         # Combine 7 ground truth + 3 cheater
      #         gt_combined_10 = np.concatenate([gt_slice_7, cheater_slice_np], axis=1)
      #         df_gt = pd.DataFrame(gt_combined_10, columns=all_var_names)
      #         df_gt.insert(0, 'Timestep', np.arange(1, n_rollout_steps + 1))
      #         df_gt.insert(0, 'DataType', 'GroundTruth')

      #         # --- 3. Combine and Save to Sheet ---
      #         df_combined = pd.concat([df_gt, df_pred])

      #         # Save to a new sheet in the same Excel file
      #         df_combined.to_excel(writer, index=False, sheet_name=f'Section_{section_num}')

      # print(f"  ✓ Successfully saved all sections.")

      print(f"\n" + "=" * 80)
      print("ALL DONE! 🎉")
      print("=" * 80)
      print(f"\nFiles saved:")
      print(f"  - {ckpt_path}")
      print(f"  - {P('training_curves.png')}")
      print(f"  - {P('autoregressive_predictions.png')}")

if __name__ == "__main__":
    # Call main
    main()

FNO 1D SPACE PIPELINE TRANSIENT MODEL INFERENCE

1. Configuration
   Dataset file: /content/gdrive/MyDrive/Colab Notebooks/FourierNeuralOperator/Transient Folder/cleaned_tensor.pt
   Input channels: 10
   Output channels: 7

2. Loading dataset...
   ✓ Dataset shape: torch.Size([50, 1279, 10, 288])
   ✓ Cases: 50
   ✓ Sections: 1279
   ✓ Variables: 10
   ✓ Timesteps: 288
   ✓ Variable names: ['PT', 'TM', 'HOL', 'HOLWT', 'UG', 'UL', 'ULWT', 'TT', 'PI', 'FI']
   ✓ Using device: cuda

3. Creating train/val/test splits...
   Total cases: 50
   Train: 40 cases (80%)
   Val: 5 cases (10%)
   Test: 5 cases (10%)

 Testing autoregressive rollout...
  Rolling out 287 timesteps...

11. Plotting SPATIAL profiles at specific timesteps...
  ✓ Saved spatial plot: /content/gdrive/MyDrive/Colab Notebooks/FourierNeuralOperator/Transient Folder/spatial_profile_t1.png
  ✓ Saved spatial plot: /content/gdrive/MyDrive/Colab Notebooks/FourierNeuralOperator/Transient Folder/spatial_profile_t30.png
  ✓ Saved sp

# IGNORE


In [ ]:
# fno_compare_inference.py
import os
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# --- your modules ---
from data_store_load import load_full_dataset
from fno_implementation_modified import FNO2DSpacetime  # uses the safe-FFT spectral layer

# --- imports you rely on (make sure these exist up top) ---
import math
from torch.utils.data import DataLoader

# =========================
# Root dir & path helpers
# =========================
try:
    DIR = dir_path  # type: ignore[name-defined]
except Exception:
    DIR = ""

def P(fn: str) -> str:
    "Join to project directory for outputs/artifacts."
    return os.path.join(DIR, fn)

def find_input(filename: str) -> str:
    cands = [
        os.path.join(DIR, filename),
        os.path.join('/mnt/data', filename),
        filename,  # absolute / custom
    ]
    for p in cands:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Could not find input file '{filename}' in {cands}")

# Ensure output dir exists (handles empty DIR too)
os.makedirs(DIR or ".", exist_ok=True)

# -------------------------
# Utilities (reuse from train)
# -------------------------
def compute_norm_stats(train_tensor: torch.Tensor):
    t = train_tensor.permute(0, 2, 1, 3).contiguous()  # [N,5,X,121]
    mean = t.mean(dim=(0, 2, 3))                       # [5]
    std  = t.std(dim=(0, 2, 3))                        # [5]
    std = torch.where(std < 1e-6, torch.ones_like(std), std)
    return mean, std

@torch.no_grad()
def make_tx_coords(T: int, X: int, section_lengths_m: np.ndarray, device=None):
    t = torch.linspace(1.0, float(T), T, device=device) / float(T)  # [T]
    sl = torch.tensor(section_lengths_m, dtype=torch.float32, device=device)  # [X]
    x = (sl - sl.min()) / (sl.max() - sl.min() + 1e-12)
    tt = t[:, None].expand(T, X)
    xx = x[None, :].expand(T, X)
    return tt[None, None], xx[None, None]  # [1,1,T,X] each

@torch.no_grad()
def predict_full_block(model: FNO2DSpacetime,
                       case_tensor: torch.Tensor,          # [X,5,121]
                       section_lengths_m: np.ndarray,
                       mean: torch.Tensor, std: torch.Tensor,
                       device: str = "cpu") -> torch.Tensor:
    """
    Returns denormalized prediction [5, 120, X] for t=1..120
    """
    model.eval()
    X = case_tensor.shape[0]
    T = 120
    # u0: [5,X]
    u0 = case_tensor[:, :, 0].permute(1, 0).contiguous()
    mean_c = mean.view(5, 1).to(device)
    std_c  = std.view(5, 1).to(device)
    u0n = (u0.to(device) - mean_c) / (std_c + 1e-8)  # [5,X]

    # inputs
    u0_tiled = u0n[:, None, :].expand(5, T, X).unsqueeze(0)     # [1,5,T,X]
    t_coord, x_coord = make_tx_coords(T, X, section_lengths_m, device=device)
    t0_mask = torch.zeros(1, 1, T, X, device=device); t0_mask[:, :, 0, :] = 1.0

    # forward
    yhat = model(u0_tiled, t_coord, x_coord, t0_mask).squeeze(0)  # [5,T,X]
    # denormalize
    yhat_denorm = yhat * std_c[:, None, :] + mean_c[:, None, :]
    return yhat_denorm  # [5,120,X]

def mae(a, b, dim=None):
    return (a - b).abs().mean(dim=dim)

def rmse(a, b, dim=None):
    return torch.sqrt(((a - b) ** 2).mean(dim=dim))

# -------------------------
# Main comparison routine
# -------------------------
def main():
    print("=" * 80)
    print("FNO TEST COMPARISON @ t=1h, 30h, 120h (3 cases, all 5 variables)")
    print("=" * 80)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # ---- paths via find_input/P ----
    dataset_file = find_input("full_dataset.pt")
    ckpt_path    = find_input("fno_full_block.pt")  # load checkpoint from known locations

    # ---- load dataset ----
    full_tensor, section_lengths, time_hours, variable_names, case_numbers = load_full_dataset(dataset_file)
    if isinstance(section_lengths, torch.Tensor):
        section_lengths = section_lengths.numpy()

    # ---- split the same way as training ----
    N = full_tensor.shape[0]
    n_train = int(0.8 * N)
    n_val   = int(0.1 * N)
    train_tensor = full_tensor[:n_train]
    val_tensor   = full_tensor[n_train:n_train+n_val]
    test_tensor  = full_tensor[n_train+n_val:]

    # ---- normalization from train ----
    mean, std = compute_norm_stats(train_tensor)

    # ---- load model ----
    model = FNO2DSpacetime(in_ch=8, out_ch=5, width=128, depth=5, k_t=8, k_x=32,
                           predict_delta_from_u0=False).to(device)
    ckpt = torch.load(ckpt_path, map_location=device, weights_only=False)
    model.load_state_dict(ckpt["model"])
    model.eval()

    # ---- pick 3 test cases ----
    num_cases = min(3, test_tensor.shape[0])
    chosen = list(range(num_cases))
    print(f"Comparing test cases indices (within test split): {chosen}")

    # ---- times of interest (1h, 30h, 120h) -> indices [0, 29, 119]
    times = {1: 0, 30: 29, 120: 119}

    # ---- gather results & errors ----
    error_rows = []
    for rel_idx, case_idx in enumerate(chosen):
        case = test_tensor[case_idx]  # [X,5,121]
        pred = predict_full_block(model, case, section_lengths, mean, std, device=device)  # [5,120,X]
        gt = case[:, :, 1:].permute(1, 2, 0).contiguous()  # [5,120,X]

        X = case.shape[0]
        x_axis = np.arange(X)

        for t_hr, t_idx in times.items():
            # One figure per case/time showing 5 variables as 5 subplots
            fig, axes = plt.subplots(5, 1, figsize=(12, 14), sharex=True)
            fig.suptitle(f"Case {case_idx} (test idx {rel_idx}) — Comparison at t = {t_hr} h")

            for v in range(5):
                y_true = gt[v, t_idx].cpu()
                y_pred = pred[v, t_idx].cpu()

                ax = axes[v]
                ax.plot(x_axis, y_true, label="Ground truth")
                ax.plot(x_axis, y_pred, label="Prediction", linestyle="--")
                ax.set_ylabel(variable_names[v] if v < len(variable_names) else f"Var{v}")
                ax.grid(True)
                if v == 0:
                    ax.legend()

                # errors across X
                v_mae = float(mae(y_pred, y_true, dim=0))
                v_rmse = float(rmse(y_pred, y_true, dim=0))
                error_rows.append({
                    "case": case_idx,
                    "time_h": t_hr,
                    "variable": variable_names[v] if v < len(variable_names) else f"Var{v}",
                    "MAE": v_mae,
                    "RMSE": v_rmse
                })

            axes[-1].set_xlabel("Spatial index (X)")
            plt.tight_layout()
            out_png = P(f"compare_case{case_idx}_t{t_hr}.png")
            plt.savefig(out_png, dpi=140)
            print(f"Saved plot: {out_png}")
            plt.close(fig)

    # ---- summary error table ----
    df = pd.DataFrame(error_rows).sort_values(["case", "time_h", "variable"])
    pd.set_option("display.max_rows", None)
    print("\nPer-case, per-time, per-variable errors (MAE, RMSE):")
    print(df.to_string(index=False))

    # Also print grouped means for a quick summary
    print("\nAverage errors grouped by time (across 3 cases):")
    print(df.groupby(["time_h", "variable"])[["MAE", "RMSE"]].mean().round(4))

    # Save CSV via P()
    csv_path = P("comparison_errors.csv")
    df.to_csv(csv_path, index=False)
    print(f"\nSaved CSV with all errors -> {csv_path}")

if __name__ == "__main__":
    main()


### IGNORE BELOW

In [ ]:
# tm_outlier_scan.py (or run in a cell)

import os
import torch
import pandas as pd
import numpy as np

# --- project helpers (same as your other scripts) ---
try:
    DIR = dir_path  # type: ignore[name-defined]
except Exception:
    DIR = ""

def P(fn: str) -> str:
    return os.path.join(DIR, fn)

def find_input(filename: str) -> str:
    cands = [os.path.join(DIR, filename), os.path.join('/mnt/data', filename), filename]
    for p in cands:
        if os.path.exists(p):
            return p
    raise FileNotFoundError(f"Could not find input file '{filename}' in {cands}")

# --- config ---
TM_UPPER_OK = 90.0   # expected max (change if needed)
TOP_K = 20           # how many top offending cases to print

# --- data load ---
from data_store_load import load_full_dataset
dataset_file = find_input("full_dataset.pt")
full_tensor, section_lengths, time_hours, variable_names, case_numbers = load_full_dataset(dataset_file)

# figure out which channel is TM robustly
try:
    tm_idx = variable_names.index("TM")
except Exception:
    tm_idx = 1  # fallback based on your order

# shapes
N, X, C, T = full_tensor.shape  # e.g., [200, 1278, 5, 121]

# pull TM only: [N, X, T]
tm = full_tensor[:, :, tm_idx, :]

# per-case max & argmax (where it happens)
flat = tm.reshape(N, -1)                       # [N, X*T]
max_vals, flat_idx = torch.max(flat, dim=1)    # [N], [N]
max_vals = max_vals.to(torch.float64)

idx_x = (flat_idx % X).to(torch.int64)         # [N]
idx_t = (flat_idx // X).to(torch.int64)        # [N]

# fraction of points > TM_UPPER_OK per case
over_mask = (tm > TM_UPPER_OK)
over_counts = over_mask.sum(dim=(1, 2)).to(torch.int64)  # [N]
total_points = torch.tensor(X * T, dtype=torch.int64)
over_frac = (over_counts.to(torch.float64) / float(X * T))  # [N]

# assemble dataframe
df = pd.DataFrame({
    "case_idx": np.arange(N, dtype=int),
    "case_number": np.array(case_numbers, dtype=int) if isinstance(case_numbers, (list, tuple)) else np.arange(N, dtype=int),
    "TM_max": max_vals.cpu().numpy(),
    "x_at_max": idx_x.cpu().numpy(),
    "t_idx_at_max": idx_t.cpu().numpy(),     # 0 = 1h, 29 = 30h, 120 = 121h if your t=0..120
    "frac_over_threshold": over_frac.cpu().numpy(),
    "count_over_threshold": over_counts.cpu().numpy(),
})

# filter offending cases
bad = df[df["TM_max"] > TM_UPPER_OK].sort_values(["TM_max", "frac_over_threshold"], ascending=False)

# save & print
out_csv = P("tm_outliers.csv")
os.makedirs(DIR or ".", exist_ok=True)
bad.to_csv(out_csv, index=False)

print(f"\nTM upper OK threshold = {TM_UPPER_OK}")
print(f"Total cases: {N} | Cases with TM_max > {TM_UPPER_OK}: {len(bad)}")
print("\nTop offending cases:")
print(bad.head(TOP_K).to_string(index=False))

print(f"\nFull list saved to: {out_csv}")
print("\nNote: t_idx_at_max is zero-based for the prediction window; if your t=0 is the initial state,")
print("      then t_idx_at_max=0 corresponds to 1h, 29 -> 30h, 119 -> 120h.")
